In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bsoup
import requests

Hi. I am using the same notebook for both questions in this assignment. 

The first part 'Data Wrangling' is where I scrape the web data and create the working dataframe. This is commented and covered in the first sumbition.

# Data Wrangling

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
data = bsoup(page.text,'html.parser')

In [14]:
postcodes = data.find_all('td')[:540:3]
borough = data.find_all('td')[1:540:3]
hood = data.find_all('td')[2:541:3]

i=0
while i < len(postcodes):
    postcodes[i] = postcodes[i].string.strip('\n')
    i=i+1
i=0
while i < len(borough):
    borough[i] = borough[i].string.strip('\n')
    i=i+1
i=0

while i < len(hood):
    if not hood[i].text is None:
        hood[i] = hood[i].text.strip('\n')        
    i=i+1

info = {'Postal Code':postcodes,\
        'Borough':borough,\
        'Neighbourhood':hood}

frame = pd.DataFrame(data=info)

frame = frame[frame['Neighbourhood'] != ''] #drop empty unasigned postal codes
frame.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Now that the initial dataset is created, we can add the gps coordinate information from the downloaded dataset.

In [5]:
geo = pd.read_csv("geocoor.csv")

Parsing.. lets see what were working with. 

In [9]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


It looks like the postal codes are ordered slightly different in this dataset. That will cause an issue if we were to just join the lists together.

So we will reorganize our Frame data to match the ordering first, then add the columns in place.

In [16]:
frame = frame.sort_values(['Postal Code'])
frame = frame.reset_index(drop=True) 
frame.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
frame[['Latitude','Longitude']] = geo[['Latitude','Longitude']]

In [18]:
frame.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Done! That was easy.